# CS_BP预处理部分

In [1]:
import numpy as np
import scipy.special as sc_special
import tensorflow as tf 
from 	tensorflow import keras
import pandas as pd
from tensorflow.keras import optimizers,metrics,layers,Sequential

In [10]:
# 导入数据以及预处理
data = pd.read_csv('GY_AIR.csv',sep=',',header=0,usecols=[1,2,3,4,5,6,7])
X = data.iloc[:430,1:]
Y = data.iloc[:430,0]
TestX = data.iloc[430:,1:]
TestY = data.iloc[430:,0]

inputnum = 6
hiddennum = 12
outputnum = 3
numsum = inputnum*hiddennum+hiddennum+hiddennum*outputnum+outputnum #6*10+10+10*3+3=103
bestfit = []
bestloss = []

def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)
    y = tf.cast(y,dtype=tf.int32)
    y = tf.one_hot(y,depth=3)
    return x,y

In [11]:
X

,so2,no2,co,o3,pm10,pm2_5
0,30,35,0.8,48,81,46
1,26,32,0.7,56,75,41
2,13,25,0.9,41,62,48
3,16,31,0.9,30,83,61
4,10,22,0.7,40,33,24
...,...,...,...,...,...,...
425,6,13,0.5,64,20,6
426,6,20,0.5,104,30,11
427,6,20,0.5,80,37,15
428,6,13,0.5,72,30,12


In [12]:
np.mean(X,axis=0)[0]

10.46046511627907

In [13]:
#数据预处理
#零均值处理
X.iloc[:,0] -= np.mean(X,axis=0)[0]  
X.iloc[:,1] -= np.mean(X,axis=0)[1]
X.iloc[:,2] -= np.mean(X,axis=0)[2]
X.iloc[:,3] -= np.mean(X,axis=0)[3]  
X.iloc[:,4] -= np.mean(X,axis=0)[4]
X.iloc[:,5] -= np.mean(X,axis=0)[5]
#归一化
X.iloc[:,0] /= np.max(np.abs(X),axis=0)[0]
X.iloc[:,1] /= np.max(np.abs(X),axis=0)[1]
X.iloc[:,2] /= np.max(np.abs(X),axis=0)[2]
X.iloc[:,3] /= np.max(np.abs(X),axis=0)[3]  
X.iloc[:,4] /= np.max(np.abs(X),axis=0)[4]
X.iloc[:,5] /= np.max(np.abs(X),axis=0)[5]

In [14]:
#零均值处理
TestX.iloc[:,0] -= np.mean(TestX,axis=0)[0]  
TestX.iloc[:,1] -= np.mean(TestX,axis=0)[1]
TestX.iloc[:,2] -= np.mean(TestX,axis=0)[2]
TestX.iloc[:,3] -= np.mean(TestX,axis=0)[3]  
TestX.iloc[:,4] -= np.mean(TestX,axis=0)[4]
TestX.iloc[:,5] -= np.mean(TestX,axis=0)[5]
#归一化
TestX.iloc[:,0] /= np.max(np.abs(TestX),axis=0)[0]
TestX.iloc[:,1] /= np.max(np.abs(TestX),axis=0)[1]
TestX.iloc[:,2] /= np.max(np.abs(TestX),axis=0)[2]
TestX.iloc[:,3] /= np.max(np.abs(TestX),axis=0)[3]  
TestX.iloc[:,4] /= np.max(np.abs(TestX),axis=0)[4]
TestX.iloc[:,5] /= np.max(np.abs(TestX),axis=0)[5]


In [15]:
batchsz = 32
db = tf.data.Dataset.from_tensor_slices((X,Y))
db = db.map(preprocess).shuffle(60000).batch(batchsz)
ds_val = tf.data.Dataset.from_tensor_slices((TestX,TestY))
ds_val = ds_val.map(preprocess).batch(batchsz)
sample = next(iter(db))
print(sample[0].shape,sample[1].shape)

(32, 6) (32, 3)


In [16]:
len(Y)

430

# CS部分

In [17]:

#n：布谷鸟数  m：布谷鸟的维度
def cuckoo_search(n, m, lower_boundary, upper_boundary, iter_num = 10,pa = 0.25, beta = 1.5, step_size = 0.1):
    """
    Cuckoo search function
    ---------------------------------------------------
    Input parameters:
        n: Number of nests
        m: Number of dimensions
        fit_func: User defined fitness evaluative function
        lower_boundary: Lower bounary (example: lower_boundary = (-2, -2, -2))
        upper_boundary: Upper boundary (example: upper_boundary = (2, 2, 2))
        iter_num: Number of iterations (default: 100) 
        pa: Possibility that hosts find cuckoos' eggs (default: 0.25)
        beta: Power law index (note: 1 < beta < 2) (default: 1.5)
        step_size:  Step size scaling factor related to the problem's scale (default: 0.1)
    Output:
        The best solution and its value
    """
    # get initial nests' locations 
    nests = generate_nests(n, m, lower_boundary, upper_boundary)
    fitness,lossness = calc_fitness( nests,0.5) #包含所有的适应度  用列表存储
    
    #得到每代最小loss
    best_loss_index = np.argmin(lossness)
    best_loss = lossness[best_loss_index]
    best_nestloss = nests[best_loss_index].copy()
    bestloss.append(best_loss)
    
    # get the best nest and record it
    best_nest_index = np.argmax(fitness)
    best_fitness = fitness[best_nest_index]
    best_nest = nests[best_nest_index].copy()
    bestfit.append(best_fitness) #全局变量，保存每代最佳适应度值
    
    best_two_nest = best_nest
    best_two_loss = lossness[best_nest_index]
    best_two_fitness = best_fitness
    print('\r\n BEST_TWO_LOSSNESS IS %.2f : \r\n',best_two_loss)
    print('\r\n BEST_TWO_FITNESS IS %.2f : \r\n',best_two_fitness)
    for _ in range(iter_num):
        nests = update_nests(lower_boundary, upper_boundary, nests, best_nest, lossness, step_size,best_fitness)
        nests = abandon_nests(nests, lower_boundary, upper_boundary, pa)
        print('\r\n*******************************************************一轮迭代后开始计算适应度*************************************************************\r\n')
        fitness,lossness = calc_fitness( nests,best_fitness)
        print('\r\n*****************************************************************结束************************************************************\r\n')
        
        min_loss_index = np.argmin(lossness)
        min_loss = lossness[min_loss_index]
        min_loss_fit = fitness[min_loss_index]
        min_nestloss = nests[min_loss_index]
        bestloss.append(min_loss)
            
        if min_loss < best_two_loss  : #and  min_loss_fit > best_two_fitness
            best_two_nest = min_nestloss
            best_two_loss = min_loss
            best_two_fitness = min_loss_fit
            print('\r\n******')
            print('\r\n BEST_TWO_FITNESS IS %.2f : \r\n',best_two_fitness)
            print('\r\n******\r\n')

    return (best_two_nest, best_two_loss,best_two_fitness)

In [18]:
def generate_nests(n, m, lower_boundary, upper_boundary):
    """
    Generate the nests' locations
    ---------------------------------------------------
    Input parameters:
        n: Number of nests
        m: Number of dimensions
        lower_boundary: Lower bounary (example: lower_boundary = (-2, -2, -2))
        upper_boundary: Upper boundary (example: upper_boundary = (2, 2, 2))
    Output:
        generated nests' locations
    """
    lower_boundary = np.array(lower_boundary)
    upper_boundary = np.array(upper_boundary)
    nests = np.empty((n, m))

    for each_nest in range(n):
        nests[each_nest] = lower_boundary + np.array([np.random.rand() for _ in range(m)]) * (upper_boundary - lower_boundary)

    return nests

In [19]:
def update_nests(lower_boundary, upper_boundary, nests, best_nest, lossness, step_coefficient,bestfitness):
    """
    This function is to get new nests' locations and use new better one to replace the old nest
    ---------------------------------------------------
    Input parameters:
        fit_func: User defined fitness evaluative function
        lower_boundary: Lower bounary (example: lower_boundary = (-2, -2, -2))
        upper_boundary: Upper boundary (example: upper_boundary = (2, 2, 2))
        nests: Old nests' locations 
        best_nest: Nest with best fitness
        fitness: Every nest's fitness
        step_coefficient:  Step size scaling factor related to the problem's scale (default: 0.1)
    Output:
        Updated nests' locations
    """
    lower_boundary = np.array(lower_boundary)
    upper_boundary = np.array(upper_boundary)
    n, m = nests.shape
    # 莱维飞行生成步长：较长时间的短步长和较短时间的长步长
    #生成的步长的shape也是n*m，保证所有元素得到更新
    #这个步长通过step_size来控制大小
    steps = levy_flight(n, m, 1.5)
    new_nests = nests.copy()

    for each_nest in range(n):
        # coefficient 0.01 is to avoid levy flights becoming too aggresive
        # and (nest[each_nest] - best_nest) 保留了最佳nest，但会导致局部最优
        step_size = step_coefficient * steps[each_nest] * (nests[each_nest] - best_nest)
        step_direction = np.random.rand(m) #0-1均匀概率分布生成方向
        #新的布谷鸟在原来的基础上加上步长和方向
        new_nests[each_nest] += step_size * step_direction  
        # apply boundary condtions
        new_nests[each_nest][new_nests[each_nest] < lower_boundary] = lower_boundary[new_nests[each_nest] < lower_boundary]
        new_nests[each_nest][new_nests[each_nest] > upper_boundary] = upper_boundary[new_nests[each_nest] > upper_boundary]

    new_fitness,new_losses = calc_fitness(new_nests,bestfitness)
    #适应度更好的才更新过去
    
    nests[new_losses < lossness] = new_nests[new_losses < lossness] 
    return nests

In [20]:
def abandon_nests(nests, lower_boundary, upper_boundary, pa):
    """
    Some cuckoos' eggs are found by hosts, and are abandoned.So cuckoos need to find new nests.
    ---------------------------------------------------
    Input parameters:
        nests: Current nests' locations
        lower_boundary: Lower bounary (example: lower_boundary = (-2, -2, -2))
        upper_boundary: Upper boundary (example: upper_boundary = (2, 2, 2))
        pa: Possibility that hosts find cuckoos' eggs
    Output:
        Updated nests' locations
    """
    lower_boundary = np.array(lower_boundary)
    upper_boundary = np.array(upper_boundary)
    n, m = nests.shape
    for each_nest in range(n):  
        #pa概率抛弃，抛弃后重新生成
        if (np.random.rand() < pa):
            #局部随机行走生成步长
            #随机两个种群相差*一个0-1的随机数
            step_size = np.random.rand() * (nests[np.random.randint(0, n)] - nests[np.random.randint(0, n)])
            nests[each_nest] += step_size
            # apply boundary condtions
            nests[each_nest][nests[each_nest] < lower_boundary] = lower_boundary[nests[each_nest] < lower_boundary]
            nests[each_nest][nests[each_nest] > upper_boundary] = upper_boundary[nests[each_nest] > upper_boundary]
    
    return nests

In [21]:
def levy_flight(n, m, beta):
    """
    This function implements Levy's flight.
    ---------------------------------------------------
    Input parameters:
        n: Number of steps 
        m: Number of dimensions
        beta: Power law index (note: 1 < beta < 2)
    Output:
        'n' levy steps in 'm' dimension
    """
    sigma_u = (sc_special.gamma(1+beta)*np.sin(np.pi*beta/2)/(sc_special.gamma((1+beta)/2)*beta*(2**((beta-1)/2))))**(1/beta)
    sigma_v = 1

    u =  np.random.normal(0, sigma_u, (n, m))
    v = np.random.normal(0, sigma_v, (n, m))

    steps = u/((np.abs(v))**(1/beta))

    return steps

In [22]:
def calc_fitness(nests,bestfitness):
    
    n, m = nests.shape
    fitness = np.empty(n)
    lossness = np.empty(n)
    for Sig_nest in range(n):
        chrom = nests[Sig_nest]
        #分解模型参数
        w1 = chrom[:inputnum*hiddennum]
        w1 = w1.reshape(inputnum,hiddennum)
        b1 = chrom[inputnum*hiddennum:inputnum*hiddennum+hiddennum]
        w2 = chrom[inputnum*hiddennum+hiddennum:inputnum*hiddennum+hiddennum+hiddennum*outputnum]
        w2 = w2.reshape(hiddennum,outputnum)
        b2 = chrom[inputnum*hiddennum+hiddennum+hiddennum*outputnum:]

        WB_l1 = (w1,b1)
        WB_l2 = (w2,b2)
        #创建模型，并赋予参数
        model = keras.Sequential([
            keras.layers.Dense(hiddennum,activation='relu',name='l1'),
            keras.layers.Dense(outputnum,name='l2')
        ])
        model.build(input_shape=[None,inputnum])
        
        #model.summary()
        model.compile(optimizer=optimizers.Adam(lr=0.01),
                    loss='mse',
                    metrics=['accuracy'])
        
        layer1 = model.get_layer('l1')
        layer2 = model.get_layer('l2')
        layer1.set_weights(WB_l1)
        layer2.set_weights(WB_l2)
        
        #训练模型
        #model.fit(db,epochs=1,validation_data=ds_val,validation_freq=1)
        model.fit(db,epochs=1,validation_data=ds_val,validation_freq=1)
        loss,acc = model.evaluate(db)  #这里是用验证集评估，其实应该用测试集吧 evaluate产生两个结果，前面是损失，后面是准确率
        lossness[Sig_nest] = loss
        fitness[Sig_nest] = acc  #将模型评估正确率作为适应度返回
        
        if(acc>bestfitness): #在计算适应度的过程当中，发现有更好的适应度就把参数保存
            #model.save_weights('my_model_fun.h5')
            bestfitness = acc
    return fitness,lossness

In [23]:
# if __name__=='__main__':
low = -3*np.ones(numsum)
upp = 3*np.ones(numsum)
best_nestloss, best_loss,fit = cuckoo_search(10,numsum, low,upp, step_size = 0.4)
best_nest = best_nestloss
best_fitness = best_loss
print('CS最优loss为:%.5f!',best_loss),print('CS最优fit为：%.5f!',fit)



14/14 [==============================] - 0s 1ms/step - loss: 51.2460 - accuracy: 0.1488

 BEST_TWO_LOSSNESS IS %.2f : 
 10.91415786743164

 BEST_TWO_FITNESS IS %.2f : 
 0.7162790894508362
14/14 [==============================] - 0s 923us/step - loss: 125.7523 - accuracy: 0.0395

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 1000us/step - loss: 51.8659 - accuracy: 0.1488

*****************************************************************结束************************************************************

14/14 [==============================] - 0s 922us/step - loss: 63.8879 - accuracy: 0.0163


14/14 [==============================] - 0s 1000us/step - loss: 68.0245 - accuracy: 0.1651

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 1ms/step - loss: 46.7989 - accuracy: 0.3419

*****************************************************************结束************************************************************


******

 BEST_TWO_FITNESS IS %.2f : 
 0.6186046600341797

******

14/14 [==============================] - 0s 1000us/step - loss: 49.8711 - accuracy: 0.1674

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 1s 24ms/step - loss: 70.7101 - accuracy: 0.3305 - val_loss: 67.9750 - val_accuracy: 0.1500


14/14 [==============================] - 0s 1ms/step - loss: 46.4349 - accuracy: 0.3442

*****************************************************************结束************************************************************

14/14 [==============================] - 0s 1ms/step - loss: 63.7346 - accuracy: 0.0233

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 1ms/step - loss: 46.6229 - accuracy: 0.3442

*****************************************************************结束************************************************************


******

 BEST_TWO_FITNESS IS %.2f : 
 0.36279070377349854

******

14/14 [==============================] - 0s 1ms/step - loss: 60.9348 - accuracy: 0.2837

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 14ms/step - lo

14/14 [==============================] - 0s 1ms/step - loss: 33.0823 - accuracy: 0.2372

*****************************************************************结束************************************************************

14/14 [==============================] - 0s 1000us/step - loss: 53.6967 - accuracy: 0.0465

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 1ms/step - loss: 33.2028 - accuracy: 0.2372

*****************************************************************结束************************************************************


******

 BEST_TWO_FITNESS IS %.2f : 
 0.33023256063461304

******

14/14 [==============================] - 0s 16ms/step - loss: 39.7868 - accuracy: 0.3415 - val_loss: 56.2361 - val_accuracy: 0.6000


14/14 [==============================] - 0s 1ms/step - loss: 51.9277 - accuracy: 0.0488

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 846us/step - loss: 33.2580 - accuracy: 0.2372

*****************************************************************结束************************************************************


******

 BEST_TWO_FITNESS IS %.2f : 
 0.5627906918525696

******

14/14 [==============================] - 0s 1ms/step - loss: 20.8936 - accuracy: 0.0953

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 777us/step - loss: 20.5791 - accuracy: 0.1116

*****************************************************************结束************************************************************

14/14 [==============================] - 1s 32ms/step -

14/14 [==============================] - 0s 768us/step - loss: 31.6645 - accuracy: 0.0116

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 799us/step - loss: 20.6992 - accuracy: 0.1047

*****************************************************************结束************************************************************


******

 BEST_TWO_FITNESS IS %.2f : 
 0.5744186043739319

******

14/14 [==============================] - 0s 691us/step - loss: 56.3293 - accuracy: 0.0000e+00

*******************************************************一轮迭代后开始计算适应度*************************************************************

14/14 [==============================] - 0s 15ms/step - loss: 60.5788 - accuracy: 0.5407 - val_loss: 79.6107 - val_accuracy: 0.3500


14/14 [==============================] - 0s 810us/step - loss: 20.6451 - accuracy: 0.1047

*****************************************************************结束************************************************************

CS最优loss为:%.5f! 0.884292721748352
CS最优fit为：%.5f! 0.5744186043739319


(None, None)

# 神经网络部分

In [130]:
chrom = best_nest
w1 = chrom[:inputnum*hiddennum]
w1 = w1.reshape(inputnum,hiddennum)
b1 = chrom[inputnum*hiddennum:inputnum*hiddennum+hiddennum]
w2 = chrom[inputnum*hiddennum+hiddennum:inputnum*hiddennum+hiddennum+hiddennum*outputnum]
w2 = w2.reshape(hiddennum,outputnum)
b2 = chrom[inputnum*hiddennum+hiddennum+hiddennum*outputnum:]

WB_layer1 = (w1,b1)
WB_layer2 = (w2,b2)

In [131]:
#用CS优化初始化阈值

model = keras.Sequential([
    keras.layers.Dense(hiddennum,name='layer1',activation='relu'),
    keras.layers.Dense(outputnum,name='layer2')
])


model.build(input_shape=[None,inputnum])
#model.summary()
model.compile(optimizer=optimizers.Adam(lr=0.01),
            loss='mse',
            metrics=['accuracy'])
#model.load_weights('my_model_fun.h5')

layer1 = model.get_layer('layer1')
layer2 = model.get_layer('layer2')
layer1.set_weights(WB_layer1)
layer2.set_weights(WB_layer2)

In [132]:
#查看模型参数
# la1 = model.get_layer('layer1')
# la2 = model.get_layer('layer2')
# (k1,b1) = la1.get_weights()
# (k2,b2) = la2.get_weights()
# k1,b1,k2,b2

In [133]:
#训练模型
model.fit(db,epochs=200,validation_data=ds_val,validation_freq=5)



Epoch 1/200
14/14 [==============================] - 0s 1ms/step - loss: 2.9714 - accuracy: 0.0948
Epoch 2/200
14/14 [==============================] - 0s 1ms/step - loss: 1.3309 - accuracy: 0.2137
Epoch 3/200
14/14 [==============================] - 0s 1ms/step - loss: 0.7095 - accuracy: 0.3881
Epoch 4/200
14/14 [==============================] - 0s 1ms/step - loss: 0.4262 - accuracy: 0.5568
Epoch 5/200
14/14 [==============================] - 0s 14ms/step - loss: 0.2818 - accuracy: 0.6266 - val_loss: 0.9001 - val_accuracy: 0.3500
Epoch 6/200
14/14 [==============================] - 0s 1ms/step - loss: 0.2312 - accuracy: 0.7066
Epoch 7/200
14/14 [==============================] - 0s 1ms/step - loss: 0.2147 - accuracy: 0.7194
Epoch 8/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1779 - accuracy: 0.7254
Epoch 9/200
14/14 [==============================] - 0s 1ms/step - loss: 0.1759 - accuracy: 0.7565
Epoch 10/200
14/14 [==============================] - 0s 6ms/step 

Epoch 150/200
14/14 [==============================] - 0s 5ms/step - loss: 0.0314 - accuracy: 0.9265 - val_loss: 0.3202 - val_accuracy: 0.6000
Epoch 151/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0345 - accuracy: 0.9147
Epoch 152/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0370 - accuracy: 0.9045
Epoch 153/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0344 - accuracy: 0.8999
Epoch 154/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0326 - accuracy: 0.9169
Epoch 155/200
14/14 [==============================] - 0s 6ms/step - loss: 0.0340 - accuracy: 0.9089 - val_loss: 0.3194 - val_accuracy: 0.6000
Epoch 156/200
14/14 [==============================] - 0s 1ms/step - loss: 0.0317 - accuracy: 0.9283
Epoch 157/200
14/14 [==============================] - 0s 1000us/step - loss: 0.0310 - accuracy: 0.9286
Epoch 158/200
14/14 [==============================] - 0s 2ms/step - loss: 0.0354 - accuracy: 0.9082
Epoc

In [134]:
# 评估模型,不输出预测结果
loss,accuracy = model.evaluate(ds_val)
print('\ntest loss',loss)
print('accuracy',accuracy)

1/1 [==============================] - 0s 24ms/step - loss: 0.3181 - accuracy: 0.6500

test loss 0.31811684370040894
accuracy 0.6499999761581421


In [135]:
#模型预测,输入测试集,输出预测结果
y_pred = model.predict(TestX)
y_pred = tf.argmax(y_pred,axis=1)
y_pred

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2],
      dtype=int64)>

In [200]:
test_y = tf.argmax(TestY_onehot,axis=1)
test_y

<tf.Tensor: shape=(20,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],
      dtype=int64)>

In [201]:
y_pred_train = model.predict(X)
y_pred_train = tf.argmax(y_pred_train,axis=1)
y_pred_train

<tf.Tensor: shape=(128,), dtype=int64, numpy=
array([2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2,
       2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2], dtype=int64)>

In [202]:
train_y = tf.argmax(Y_onehot,axis=1)
train_y

<tf.Tensor: shape=(128,), dtype=int64, numpy=
array([2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2,
       2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1,
       1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2, 0, 0, 2, 2, 1, 2, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2], dtype=int64)>

# 绘图部分

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
bestloss

In [ ]:
sns.set_style("ticks")
x = list(range(0,len(bestfit)))
sns.lineplot(x,bestfit)


In [ ]:
error = y_pred - TestY_onehot
errorArr = []
for i in range(len(error)):
    errorArr.append(int(sum(abs(error[i]))))
sns.lineplot(list(range(0,len(errorArr))),errorArr,color="b")



In [ ]:
sns.lineplot(x,bestloss,color="r")

In [ ]:
#sns主题风格  darkgrid  whitegrid  dark  white  ticks
sns.set_style("ticks")  #设置主题风格
sns.color_palette("hls",8)  #设置颜色空间种类（几种可用颜色）
data=np.random.normal(size=(20,8)) + np.arange(8) /2
sns.boxplot(data = data,palette = sns.color_palette("hls",8))

# 描述两个变量的关系 最好用散点图

In [ ]:
import pandas as pd
mean,cov = [0,1],[(1,.5),(.5,1)]
data = np.random.multivariate_normal(mean,cov,200)
df = pd.DataFrame(data,columns=["x","y"])
#绘制散点图
sns.jointplot(x="x",y="y",data = df,color="r")  #如果点很多，用颜色深度表示数量 kind="hex" ,可以单独传x y

# 移植STM32准备

In [ ]:
Testx = X / X.max().max()
Testx = np.array(Testx)

In [ ]:
aa = []
aa.append(list(Testx[0]))
aa

In [ ]:
pre_sig=model.predict(aa)
pre_sig

In [ ]:
index = np.argmax(pre_sig,axis=None)
index

In [ ]:
Y

# 模型保存

In [ ]:
model.save('Env.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(load_model)
tflite_model = converter.convert()
open("level_check.tflite", "wb").write(tflite_model)

In [ ]:
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D
from tensorflow.keras import Sequential
import tensorflow.keras as keras
import numpy as np

In [ ]:


# #建立一个测试模型
model = Sequential([
    Dense(4, input_shape=(5,), name='dense_xiaoming',
          kernel_initializer='zeros',  # 全部初始化为0
          bias_initializer='ones'),  # 全部初始化为1
])



In [ ]:
x = np.ones(shape=(8, 5))  # 创建测试数据
y = model(x)
layer = model.get_layer('dense_xiaoming')  # 通过层的名字得到层
(k, b) = layer.get_weights()  # 查看层的初始化权重值和偏置项
print(k)
print(b)

In [ ]:
layer